# CDO bindings test

Before using the cdo bindigs we need to import the cdo bindings package (`python-cdo` in `conda` or `mamba`).
Additionally, before calling an operator, an object has to be created first.

We make use of the `tempdir` argument in order to store temporary data output in a custom folder and avoid heavy memory usage.

In [1]:
import cdo
import tools

tempPath = '/scratch/users/mnurisso/tempcdo/'
cdo = cdo.Cdo(tempdir=tempPath)

In [2]:
# Load some test data

path = '/work/datasets/obs/ERA5/mean_sea_level_pressure/mon/'
filename = 'ERA5_mean_sea_level_pressure_mon_0.25x0.25_sfc_1979-2021.nc'

infile = path+filename
print(infile)

/work/datasets/obs/ERA5/mean_sea_level_pressure/mon/ERA5_mean_sea_level_pressure_mon_0.25x0.25_sfc_1979-2021.nc


## Simple command test

We try to run a simple `showname` cdo command.

In [3]:
cdo.showname(input=infile)

['msl']

## NAO index cdo pipeline

We evaluate the NAO index in a similar way of what is the function `station_based_index()` in the `index.py` file.
The result of the analysis has to be compared with what is obtained with the `station_based_index()` function in order to test the function itself.

### Anomalies

In [4]:
msl_ma = cdo.monmean(input=infile)
msl_ma_av = cdo.ymonmean(input=msl_ma)
msl_an = cdo.sub(input=[msl_ma, msl_ma_av])

In [11]:
test = cdo.sub(input=[msl_ma,msl_ma_av],returnXDataset='test')

In [12]:
test

<xarray.Dataset>
Dimensions:    (time: 516, bnds: 2, lon: 1440, lat: 721)
Coordinates:
  * time       (time) datetime64[ns] 1979-01-01 1979-02-01 ... 2021-12-01
  * lon        (lon) float64 0.0 0.25 0.5 0.75 1.0 ... 359.0 359.2 359.5 359.8
  * lat        (lat) float64 90.0 89.75 89.5 89.25 ... -89.25 -89.5 -89.75 -90.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    msl        (time, lat, lon) float32 ...
Attributes:
    CDI:          Climate Data Interface version 2.1.1 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    institution:  European Centre for Medium-Range Weather Forecasts
    history:      Tue Feb 14 15:13:24 2023: cdo -O -s -f nc -sub /scratch/use...
    frequency:    mon
    CDO:          Climate Data Operators version 2.1.1 (https://mpimet.mpg.de...

Commands can also be concatenated like `cdo.timmean(input= '-monmean' + infile)`

### Interpolation to NAO coordinates

In [19]:
# Get the latitude and longitudes for NAO

diagname  = 'teleconnections'
telecname = 'NAO'

namelist = tools.load_namelist(diagname)

lon1 = tools.lon_180_to_360(namelist[telecname]['lon1'])
lat1 = namelist[telecname]['lat1']
lon2 = tools.lon_180_to_360(namelist[telecname]['lon2'])
lat2 = namelist[telecname]['lat2']

# Interpolate to the acquired coordinates

field_an1 = cdo.remapnn("lon={0}_lat={1}".format(lon1,lat1),input=msl_an)
field_an2 = cdo.remapnn("lon={0}_lat={1}".format(lon2,lat2),input=msl_an)

### Evaluate NAO index

In [20]:
# Rolling average

months = 3
field_an1_ma = cdo.runmean("{0}".format(months),input=field_an1)
field_an2_ma = cdo.runmean("{0}".format(months),input=field_an2)

In [21]:
# NAO (not index)

diff_ma = cdo.sub(input=[field_an1_ma, field_an2_ma])
mean_ma = cdo.timmean(input=diff_ma)
std_ma = cdo.timstd(input=diff_ma)

In [22]:
# Index NAO
sub_nao = cdo.sub(input=[diff_ma,mean_ma])

# setup output
machine = 'wilma'
config = tools.load_config(machine)

ofile  = config[diagname]['output']
ofile += '/cdo_nao.nc'

cdo.div(input=[sub_nao,std_ma],output=ofile)

'/scratch/users/mnurisso/aqua/teleconnections/cdo_nao.nc'

## Cleanup the temporary files

Temporary files are not deleted when the jupyter notebook is closed. 
In order to empty the space we need to run an additional command.

In [9]:
cdo.cleanTempDir()

## ENSO cdo pipeline

We evaluate ENSO 3.4 in a similar way of what is the function `regional_mean_index()` in the `index.py` file.
The result of the analysis has to be compared with what is obtained with the `regional_mean_index()` function in order to test the function itself.

In [10]:
path_ENSO = '/work/datasets/obs/ERA5/skin_temperature/mon/'
filename_ENSO = 'ERA5_skin_temperature_mon_0.25x0.25_sfc_1979-2019.nc'

infile_ENSO = path_ENSO+filename_ENSO
print(infile_ENSO)

telecname = 'ENSO'

/work/datasets/obs/ERA5/skin_temperature/mon/ERA5_skin_temperature_mon_0.25x0.25_sfc_1979-2019.nc


In [11]:
lonW = tools.lon_180_to_360(namelist[telecname]['lonW'])
latN = namelist[telecname]['latN']
lonE = tools.lon_180_to_360(namelist[telecname]['lonE'])
latS = namelist[telecname]['latS']
print(lonW,lonE,latN,latS)

190 240 5 -5


In [13]:
t_sel = cdo.sellonlatbox('{},{},{},{}'.format(lonW,lonE,latS,latN),input=infile_ENSO)
t_mean = cdo.fldmean(input=t_sel)

In [15]:
# Rolling average
months = 3

# setup output
ofile_ENSO  = config[diagname]['output']
ofile_ENSO += '/cdo_enso.nc'

t_ma = cdo.runmean("{0}".format(months),input=t_mean,output=ofile_ENSO)